# Game of Life

## Installation

In [1]:
!pip install mesa

     |████████████████████████████████| 668 kB 25.6 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


## Libraries

In [2]:
# Author: Lourdes Badillo
"""
Basic game of life model with agents
"""

from mesa import Agent, Model
from mesa.space import SingleGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

## Classes

In [12]:
def get_grid(model):
  grid = np.zeros((model.grid.width, model.grid.height))
  for (content, x, y) in model.grid.coord_iter():
    grid[x][y] = content.state

  return grid
    
class Cell(Agent):
  """A cell(agent) that can be either death (0) or alive (1)"""

  def __init__(self, unique_id, model):
      super().__init__(unique_id, model)
      self.state = np.random.choice([0,1])
      self.next_state = None

  def step(self):
      """State Machine definition"""
      neighbors = self.model.grid.get_neighbors(
          self.pos, 
          moore = True, 
          include_center = False

      )
      alive = 0
      for neighbor in neighbors: 
        if neighbor.state is not None: 
          alive = alive + neighbor.state
      
        if alive < 2 or alive >3: 
          self.next_state = 0 #rip
        
        if alive == 3: 
          self.next_state = 1 #it's aliveee
        
        self.state = self.next_state

  def advance(self):
      self.state = self.next_state

class GOLModel(Model):
  """A game of life model with some number of cells (agents)."""
  def __init__(self, size):
      self.num_agents = size
      self.grid = SingleGrid(size, size, True)
      self.schedule = SimultaneousActivation(self)
      # Create agents
      for (content, x, y) in self.grid.coord_iter():
          a = Cell((x,y), self)
          self.grid.place_agent(a,(x,y))
          self.schedule.add(a)

      self.datacollector = DataCollector(
          model_reporters = {'Grid': get_grid}
      )
  def step(self):
    self.datacollector.collect(self)
    self.schedule.step()

## Main

In [13]:
size = 20 
num_gen = 40

model = GOLModel(size)

for i in range(num_gen):
  model.step()


In [16]:
all_grid = model.datacollector.get_model_vars_dataframe()



In [17]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
    patch.set_data(all_grid.iloc[i][0])

In [19]:
anim = animation.FuncAnimation(fig, animate, frames=num_gen)

anim